In [1]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load a pre-trained SentenceTransformer model for embeddings
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # A lightweight embedding model

# Load a pre-trained language model for generating text
generator = pipeline('text-generation', model='gpt2')  # Use GPT-2 for local text generation


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [20]:
import socket
from pymilvus import connections

try:
    connections.connect("default", host="localhost", port="19530", timeout=10)
    print("Connected to Milvus")
except Exception as e:
    print(f"Failed to connect to localhost: {e}")



Connected to Milvus


In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# Load model and tokenizer
model_name = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    load_in_8bit=True if device == "cuda" else False
)

# Create a text-generation pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    device=device
)

# Create a LangChain LLM from the pipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Set up the embedding model
embeddings = HuggingFaceEmbeddings(model_name="paraphrase-MiniLM-L6-v2")

# Create Milvus vector store (assuming you've already added your documents)
vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="research_papers"
)

# Create a retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Function to handle queries
def answer_query(query):
    result = qa_chain({"query": query})
    answer = result['result']
    sources = [doc.metadata['source'] for doc in result['source_documents']]
    
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print("Sources:")
    for source in set(sources):
        print(f"- {source}")



pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [32]:
# Example usage
answer_query("How does the naive bayes approach work?")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: How does the naive bayes approach work?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Based on the above shortcomings, this paper improves the naive Bayes classification
algorithm by combining feature weighting and Laplace calibration. The improved naive
Bayes classification algorithm can overcome the above shortcomings and make full use
of the information of the training set to greatly improve the accuracy of the original
naive Bayes classification algorithm. At the same time, the improved naive Bayes classi-
fication algorithm is applied to the scene of traffic risk management to effectively pre-
dict and classify the driver ’s driving risk and finally implement effective risk
management.Chen et al. EURASIP Journal on Advances in Signal Processing         (2021) 2021:30 Page 2 of 12

The rest of the paper is organized as follows: the improved naive Baye

In [ ]:
answer_query("How do I make RAG more efficient?")